In [1]:
from transformers import AutoTokenizer
import nltk
import datasets
from tqdm import tqdm
import torch
import pandas as pd
import pyarrow as pa
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\braim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")
train_data = pd.DataFrame(train_data)
val_data = pd.DataFrame(val_data)
test_data = pd.DataFrame(test_data)
for index, text in enumerate(train_data['article']):
    if len(text) % 512 > 1.5:
        train_data.loc[index,'article'] = ''.join(list(text)[256:]) + ''.join(list(text)[-256:])
    elif len(text) > 512:
         train_data.loc[index,'article'] = ''.join(list(text)[-512:])
    else:
        continue
for index, text in enumerate(val_data['article']):
    if len(text) % 512 > 1.5:
        val_data.loc[index,'article'] = ''.join(list(text)[256:]) + ''.join(list(text)[-256:])
    elif len(text) > 512:
         val_data.loc[index,'article'] = ''.join(list(text)[-512:])
    else:
        continue
for index, text in enumerate(test_data['article']):
    if len(text) % 512 > 1.5:
        test_data.loc[index,'article'] = ''.join(list(text)[256:]) + ''.join(list(text)[-256:])
    elif len(text) > 512:
         test_data.loc[index,'article'] = ''.join(list(text)[-512:])
    else:
        continue
train_data = datasets.Dataset(pa.Table.from_pandas(train_data))
val_data = datasets.Dataset(pa.Table.from_pandas(val_data))
test_data = datasets.Dataset(pa.Table.from_pandas(test_data))

Found cached dataset cnn_dailymail (C:/Users/braim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
Found cached dataset cnn_dailymail (C:/Users/braim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
Found cached dataset cnn_dailymail (C:/Users/braim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


In [3]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')
prefix = 'summarize:'
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

C:\Users\braim\AppData\Roaming\Python\Python38\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [56]:
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function,batched=True)

  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [4]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = AutoModelForSeq2SeqLM.from_pretrained("./T5base/t5-base/256_tokens").to(device)

In [58]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [59]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    gradient_accumulation_steps=20,
    eval_steps=500,
    save_steps=1000,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [60]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, highlights, article. If id, highlights, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\braim\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287113
  Num Epochs = 2
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 60
  Gradient Accumulation steps = 20
  Total optimization steps = 9570
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, usi

Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, highlights, article. If id, highlights, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13368
  Batch size = 3
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, highlights, article. If id, highlights, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13368
  Batch size = 3
Saving model checkpoint to ./t5results\checkpoint-1000
Configuration saved in ./t5results\checkpoint-1000\config.json
Model weights saved in ./t5results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./t5results\checkpoint-1000\tokenizer_config.json
Special to

TrainOutput(global_step=9570, training_loss=2.2067637715593773, metrics={'train_runtime': 16424.6028, 'train_samples_per_second': 34.961, 'train_steps_per_second': 0.583, 'total_flos': 7.76878419370967e+16, 'train_loss': 2.2067637715593773, 'epoch': 2.0})

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained('./T5base/t5-base/256_tokens').to(device)

In [6]:
from transformers import pipeline
text_generator = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    framework="pt",
    max_length=50,
    device=0
)

In [ ]:
#model.save_pretrained('./T5base/t5-base/256_tokens')

In [10]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(metric):
    article_batches = list(generate_batch_sized_chunks(test_data['article'],1))
    target_batches = list(generate_batch_sized_chunks(test_data['highlights'],1))
    for article, target in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        generated_summary = text_generator(article)
        metric.add_batch(predictions=generated_summary, references=target)
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [8]:
import math
print(f"Perplexity: {math.exp(2.21):.2f}")

Perplexity: 9.12


In [11]:
from datasets import load_metric
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')
score = calculate_metric_on_test_ds(rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
pd.DataFrame(rouge_dict, index = ['fine_tuned T5-small'])

  0%|          | 9/11490 [00:05<1:33:58,  2.04it/s]C:\Users\braim\AppData\Roaming\Python\Python38\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 11490/11490 [1:35:45<00:00,  2.00it/s]


,rouge1,rouge2,rougeL,rougeLsum
fine_tuned T5-small,0.339112,0.149151,0.24515,0.296455
